In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Fri Jun  5 16:15:03 PDT 2020


In [2]:
output_dirs = ['/ihme/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/v6.0_prelim_maternal_iron_lri_bp/india_5_20/count_data/']

locations = ['India']

In [5]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,232,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,232,folic_acid_fortification_scale_up,0.0


In [6]:
np.unique(pt['folic_acid_fortification_group'])

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [7]:
np.unique(pt['vitamin_a_fortification_group'])

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

# Vitamin A Fortification Coverage

In [15]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR

data = pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario != 'folic_acid_fortification_scale_up'] # look only at baseline and vitamin a scenario
covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

overall = data.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / overall
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                         year                                        
baseline                         2020   10.0  0.239075  0.005391  0.231401   
                                 2021   10.0  0.238576  0.005671  0.230036   
                                 2022   10.0  0.238706  0.005918  0.229512   
                                 2023   10.0  0.238736  0.005744  0.229434   
                                 2024   10.0  0.238746  0.005735  0.229425   
vitamin_a_fortification_scale_up 2020   10.0  0.805017  0.003962  0.798885   
                                 2021   10.0  0.896913  0.003362  0.891352   
                                 2022   10.0  0.921161  0.003103  0.916339   
                                 2023   10.0  0.925528  0.003118  0.920607   
                                 2024   10.0  0.927869  0.003005  0.923880   

                                            25%       50%       75%       max  
scenario                         year                                          
baseline                         2020  0.234630  0.241222  0.242718  0.246571  
                                 2021  0.234080  0.240239  0.242426  0.247111  
                                 2022  0.233835  0.240690  0.243083  0.246337  
                                 2023  0.234610  0.241022  0.242870  0.245666  
                                 2024  0.234315  0.241391  0.243131  0.245133  
vitamin_a_fortification_scale_up 2020  0.802422  0.806020  0.807884  0.809864  
                                 2021  0.894741  0.897516  0.899499  0.901217  
                                 2022  0.919175  0.921573  0.923550  0.924976  
                                 2023  0.923741  0.925890  0.927965  0.929326  
                                 2024  0.925568  0.927822  0.930152  0.932075

## Vitamin A coverage conclusion


- coverage is starting in 2020 still, not 2021, although the coverage value that intervention scenario doesn't make sense to me
- based on the coverage algorithm, the 2021 coverage in the intervention scenario should be 89%, which it is.
- not sure why 2020 intervention scenario coverage in 2020 is 80%... this is outside of the confidence interval for parameter b in the coverage algorithm

# Folic Acid Fortification Coverage

In [19]:
# COMPUTE COVERAGE PROPORTION BY SCENARIO, YEAR - Folic acid

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario != 'vitamin_a_fortification_scale_up']
covered = data.loc[data.folic_acid_fortification_group == 'covered']
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

overall = data.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / overall
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                          year                                        
baseline                          2020   10.0  0.002894  0.000237  0.002582   
                                  2021   10.0  0.011849  0.000861  0.010786   
                                  2022   10.0  0.024229  0.001890  0.022072   
                                  2023   10.0  0.036711  0.002934  0.033422   
                                  2024   10.0  0.049466  0.003994  0.044538   
folic_acid_fortification_scale_up 2020   10.0  0.002887  0.000263  0.002549   
                                  2021   10.0  0.011880  0.000929  0.010819   
                                  2022   10.0  0.024735  0.002040  0.022638   
                                  2023   10.0  0.046666  0.003218  0.043032   
                                  2024   10.0  0.083022  0.004620  0.077059   

                                             25%       50%       75%       max  
scenario                          year                                          
baseline                          2020  0.002734  0.002856  0.003067  0.003297  
                                  2021  0.011154  0.011752  0.012281  0.013196  
                                  2022  0.022683  0.023917  0.025726  0.027315  
                                  2023  0.034174  0.036344  0.039227  0.041050  
                                  2024  0.045948  0.049191  0.053040  0.055072  
folic_acid_fortification_scale_up 2020  0.002677  0.002895  0.003015  0.003386  
                                  2021  0.011121  0.011596  0.012524  0.013247  
                                  2022  0.022820  0.024218  0.026564  0.027825  
                                  2023  0.043857  0.045845  0.049900  0.051245  
                                  2024  0.079347  0.081954  0.087220  0.089058

- unexpected results when including folic_acid_fortification_group == 'unknown' values
- try again by excluding unknown coverage...

In [18]:
# COMPUTE COVERAGE PROPORTION BY SCENARIO, YEAR - Folic acid

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario != 'vitamin_a_fortification_scale_up']
covered = data.loc[data.folic_acid_fortification_group == 'covered']
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

known_coverage = data.loc[data.folic_acid_fortification_group != 'unknown']
known_coverage = known_coverage.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / known_coverage
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                          year                                        
baseline                          2020   10.0  0.061913  0.005072  0.054974   
                                  2021   10.0  0.062308  0.004454  0.056717   
                                  2022   10.0  0.062464  0.004834  0.056847   
                                  2023   10.0  0.062030  0.004902  0.056472   
                                  2024   10.0  0.061962  0.004985  0.055790   
folic_acid_fortification_scale_up 2020   10.0  0.061988  0.005636  0.054976   
                                  2021   10.0  0.062575  0.004918  0.056894   
                                  2022   10.0  0.063808  0.005320  0.058288   
                                  2023   10.0  0.078798  0.005478  0.072584   
                                  2024   10.0  0.103964  0.005884  0.096464   

                                             25%       50%       75%       max  
scenario                          year                                          
baseline                          2020  0.058483  0.061143  0.066044  0.069406  
                                  2021  0.058772  0.061780  0.064760  0.068940  
                                  2022  0.058613  0.061489  0.066280  0.070305  
                                  2023  0.057764  0.061282  0.066288  0.069271  
                                  2024  0.057530  0.061522  0.066455  0.069017  
folic_acid_fortification_scale_up 2020  0.057879  0.061803  0.065073  0.072784  
                                  2021  0.058680  0.061022  0.066031  0.069919  
                                  2022  0.059023  0.062268  0.068503  0.071822  
                                  2023  0.074186  0.077214  0.084287  0.086546  
                                  2024  0.099376  0.102508  0.109362  0.111799

## Folic Acid Fortification Coverage Conclusion
- Looks as expected based on coverage algorithm, although differences are slightly hard to discern due to very close values for parameters a and b in algorithm
